In [90]:
%pip install -qU openai tqdm

Note: you may need to restart the kernel to use updated packages.


In [91]:
MODELS = [
    "google/gemma-3-4b-it",
    "google/gemma-3-27b-it",
    "gpt-5"
]


# Load dataset categories from JSON file
DATASET1_NAMES = [
    "minecraft",
    "ucla",
    "nostupidquestions",
    "copypasta",
    "varietypack",
]
DATASETS1 = [
    {
        "minecraft": 1,  
    },
    {
        "ucla": 1,  
    },
    {
        "nostupidquestions": 1,  
    },
    {
        "copypasta": 1,  
    },
    {
        "nerdy": 1,  
        "personal": 1,
        "amitheasshole": 1,
        "tech": 1,
        "pop": 1,
        "animals": 1, 
        "boomerhumor": 1,
        "copypasta": 1,
        "creative": 1,
        "food": 1,
        "nba": 1,
        "religion": 1,
        "school": 1,
        "ucla": 1,
    },
]

DATASET2_NAMES = [
    "nerdy",
    "personal",
    "unalike",
    "alike",
    "formatspecific",
    "college",
    "newmother",
]
DATASETS2 = [
    {
        "nerdy": 1,  
    },
    {
        "personal": 1,  
    },
    { # unalike
        "pop": 1,  
        "religion": 1,
        "tech": 1
    },
    { # alike
        "tech": 1,
        "nerdy": 1,
        "finance": 1,
    },
    { # format specific
        "copypasta": 1,
        "nostupidquestions": 1,
        "amitheasshole": 1,
    },
    { # college student
        "ucla": 1,
        "nerdy": 1,
        "okbuddy": 1,
        # "copypasta": 1,
        "pop": 1,
        "food": 1,
        "animals": 1,
    },
    { # new mother
        "pregnancy": 1,
        "parenting": 1,
        "baby": 1,
        "food": 1,
        "amitheasshole": 1,
        "pop": 1,
        "boomerhumor": 1,
    },
]

TRAIN_SIZES = [
    10, 20, 50, 100, 250, 500, 1000,2000
]

EXPERIMENTS = [
    "self defined",
    "summary",
    "like history",
    # "fine tune",
    "soft prompt",
]

PROMPT_TOKENS = 64
MICRO_BATCH_SIZE = 1
GRAD_ACCUM_STEPS = 1
LEARNING_RATE = 0.2
NUM_TRAIN_STEPS = 1000  
MAX_SEQ_LEN = 2048

MODEL_OUTPUT_DIR = "models"
GENERATED_OUTPUT_DIR = "generated"

In [92]:
from typing import List, Dict
import json
import re
import random, math

def load_datasets_proportional_objects(datasets_dict: Dict[str, float], total_posts: int) -> List[dict]:
    examples: List[dict] = []
    
    # Get total of all values in datasets_dict
    total_proportion = sum(datasets_dict.values())
    for dataset_name, proportion in datasets_dict.items():
        # Calculate number of posts for this dataset
        factor = proportion / total_proportion
        target_count = math.ceil(total_posts * factor)
        # print(f"Loading {target_count} posts from {dataset_name} dataset ({factor*100:.1f}%)")
        
        # Load sampled Reddit posts from JSON created by sample-posts.py
        # Each item is a dict with keys: title, subreddit, self_text
        try:
            with open(f"../../../datasets/{dataset_name}.json", "r", encoding="utf-8") as f:
                reddit_posts: List[dict] = json.load(f)
        except FileNotFoundError:
            print(f"Warning: Could not find dataset file for {dataset_name}")
            continue
        
        # Filter valid posts (must have self_text and no image_url)
        valid_posts = []
        for p in reddit_posts:
            title = p.get("title", "")
            self_text = p.get("self_text", "")
            subreddit = p.get("subreddit", "")
            image_url = p.get("image_url", "")
            
            if self_text and not image_url:
                valid_posts.append({
                    "title": title,
                    "self_text": self_text,
                    "subreddit": subreddit,
                })
        
        # print(f"Found {len(valid_posts)} valid posts in {dataset_name}")
        
        # Sample the target number of posts
        if len(valid_posts) >= target_count:
            # Randomly sample target_count posts
            sampled_posts = random.sample(valid_posts, target_count)
        else:
            # Use all available posts if we don't have enough
            print(f"Warning: Only {len(valid_posts)} posts available, using all")
            sampled_posts = valid_posts
        
        examples.extend(sampled_posts)
    
    # Shuffle the final dataset to mix posts from different datasets
    random.shuffle(examples)
    
    # print(f"Loaded dataset {datasets_dict} with {total_posts} posts")
    return examples

In [93]:
prompt_data = load_datasets_proportional_objects({ "minecraft": 1 }, 5)

prompt_data_str = "".join(
    f"title: {p['title']}\n self_text: {p['self_text']}\n subreddit: {p['subreddit']}\n\n"
    for p in prompt_data
)
prompt_data_str


'title: lag issue\n self_text: Anyone else have this issue with bedrock where while you are holding the map on your offhand the game lags\n subreddit: Minecraft\n\ntitle: Can my brother with a Mac no longer play Minecraft with us?\n self_text: It’s saying only pc is this really true?\n subreddit: Minecraft\n\ntitle: How do i get rid of pillager raid symbol.\n self_text: I killed some pillagers now the raid wont stop and they wont stop sounding the horn what do i do to make it stop.\n subreddit: Minecraft\n\ntitle: Ray tracing for console\n self_text: Is there any new information for Ray tracing coming to next gen consoles\n subreddit: Minecraft\n\ntitle: Looking for a texture pack\n self_text: Anyone know any good texture packs that make the trees appear as cherry blossoms. Or just any aesthetic texture packs that prioritise the colours pink/purple?\n subreddit: Minecraft\n\n'

In [106]:
# judge whether each generated post adheres to dataset category, heuristic based on word content and llm judge

from openai import OpenAI
import os, dotenv
from functools import reduce

dotenv.load_dotenv()

client = OpenAI()

def judge_post(post, dataset):
    posts = load_datasets_proportional_objects(dataset, 100)
    posts_str = "".join(
        f"title: {p['title']}\n self_text: {p['self_text']}\n subreddit: {p['subreddit']}\n\n"
        for p in posts
    )
    
    SYS_PROMPT = """\
# Role and Objective
- Act as a judge to evaluate Reddit posts for similarity and appropriateness relative to a given dataset.

# Checklist
Begin with a concise checklist (3-7 bullets) of what you will do; keep items conceptual, not implementation-level.

# Instructions
- Receive a single Reddit post and a dataset of posts as context.
- Assess if the post's style, content, and coherence align with the patterns in the dataset, paying special attention to common subreddit formats and conventions.
- ignore any tokens such as <eos>

## Sub-categories
- **style_adherence**: Rate as 1 if the writing style matches the dataset (including format conventions typical for specific subreddits, e.g., AITA, NoStupidQuestions); otherwise 0.
- **content_adherence**: Rate as 1 if the content aligns with the dataset and the post's subreddit; otherwise 0. If the subreddit is right but the main content is not, set to 0.
- **coherence**: Rate as 1 if the post is as coherent as typical posts in the dataset; otherwise 0.

# Context
- Example dataset and posts provided for reference.
- Enforce strict field presence: each post must have a valid title, self_text, and subreddit.
- If any required fields are missing or malformed, output all values as 0.

# Reasoning Steps
- Review all fields for presence and valid form.
- Compare post with dataset patterns in style, content, and logical coherence.

# Planning and Verification
- Require: title, self_text, and subreddit fields.
- Validate the JSON object fields and assess individual criteria stepwise.
- After making your assessment, validate that the output matches the specification and self-correct if necessary.

# Output Format
Respond only with a JSON object in this structure (no other text):
```json
{
  "style_adherence": 0 or 1,
  "content_adherence": 0 or 1,
  "coherence": 0 or 1
}
```
# Output Format (Specification)
Include exactly the following fields as integer 0 or 1 values (except the explanation which can be a string):
- style_adherence
- content_adherence
- coherence
- explanation

# Verbosity
- Output must be concise: strictly a single JSON object.

# Stop Conditions
- Do not output any text outside the JSON object.
- If any post field is missing or malformed, return all-zeroes JSON.

"""

    USER_PROMPT = f"""
Here's the dataset:

{posts_str}

Here's the post you need to judge:

{post}
"""

    response = client.responses.create(
        model="gpt-5-nano",
        instructions=SYS_PROMPT,
        input=USER_PROMPT,
    )
    
    # print("Judged post for dataset: ", dataset)
    
    return response.output_text


    
# load in 100 posts from dataset and judge whether the generated post adheres to the style and content of the dataset as well as the overall coherence of the generated post relative to the dataset

# load a handful of psots from each generated file and see how often i agree with the llm judgement (obviously I need to judge before the llm judgement is shown)

# three judgements:
# 1. does the post adhere to the style of the dataset?
# 2. does the post adhere to the content of the dataset?
# 3. is the post coherent?

# in prompt show example dataset and generated posts as well as my judgements



In [108]:
import json

post = """title: I need a name for my son,
    self_text: My wife and I are having a baby, and we're trying to decide on a name for our boy. We want something that is strong and traditional but also unique and modern. We have been going back and forth with names like Henry, George, William, James, Jack, Charlie, Finn, Owen, Leo, Caleb, Noah, Jude, Jasper, Silas, Luke, Ethan, Benjamin, Samuel, David, Alexander, Daniel, Arthur, Charles, Oliver, Theodore, Edward, Michael, Joseph, Isaac, John, Peter, Matthew, Thomas, Robert, Paul, Richard, Justin, Ryan, Jason, Brian, Christopher, Nicholas, Jacob, Logan, Aiden, Connor, Jackson, Wyatt, Julian, Tyler, Dalton, Grayson, Lincoln, Winston, Milo, Levi, Silas, Asher, Ezra, Zion, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude, Jude
    subreddit: """
dataset = DATASETS1[3]
judgement = json.loads(judge_post(post, dataset))

judgement

{'style_adherence': 0,
 'content_adherence': 0,
 'coherence': 0,
 'explanation': 'Missing or empty subreddit field; required fields not properly provided.'}

## Judgement Sample for Agreement Rate (100 posts across each all the files)

In [96]:
# load generated posts
def load_generated_posts(ablation, model, dataset_name, train_size):
    with open(f"../generated/{ablation}/{model}/{dataset_name}/{train_size}.json", "r") as f:
        return json.load(f)



In [112]:
import json

post = load_generated_posts("train_size_ablation", MODELS[0], "minecraft", 100)[2]
post = f"title: {post['title']}\n self_text: {post['self_text']}\n subreddit: {post['subreddit']}\n\n"
print(post)
dataset = DATASETS1[0]
judgement = json.loads(judge_post(post, dataset))

judgement

title: Can anyone else have trouble with my Xbox?
 self_text: It keeps saying that it cant connect to the server I'm on. I made sure everything was working, but now I just get a message saying that it cant connect to the server. My friends are able to connect fine so I'm wondering if its something specific to me.
 subreddit: Minecraft<eos>




{'style_adherence': 1,
 'content_adherence': 1,
 'coherence': 1,
 'explanation': '- Checklist:\n  - Verify required fields are present and valid (title, self_text, subreddit), handling the <eos> token if present.\n  - Compare post style to dataset conventions (reddit/minecraft Q&A format).\n  - Compare content to dataset topics (Minecraft crossplay/connectivity issues).\n  - Assess coherence and readability vs. dataset norm.\n- Assessment: style_adherence=1, content_adherence=1, coherence=1. The post is a clear Minecraft connectivity question typical of the dataset.'}

In [113]:
judgements = []

In [ ]:
from tqdm.auto import tqdm
import json

for model in tqdm(MODELS, desc="Models"): # cost: about $5 for 120 judgements
    if model == "gpt-5": continue # skip gpt-5
    for dataset in tqdm([{"minecraft": 1}, {"copypasta": 1}, {"ucla": 1}], desc="Datasets"):
        dataset_name = list(dataset.keys())[0]
        for train_size in tqdm([10, 100], desc="Train Sizes"):
            generated_posts = load_generated_posts("train_size_ablation", model, dataset_name, train_size)[:10]
            
            for post in tqdm(generated_posts, desc="Posts"):
                post = f"title: {post['title']}\n self_text: {post['self_text']}\n subreddit: {post['subreddit']}\n\n"
                judgement = json.loads(judge_post(post, dataset))
                print(post)
                print(judgement)
                judgements.append(judgement)

In [ ]:
print(judgements)

In [115]:
posts = []
for model in MODELS:
    if model == "gpt-5": continue # skip gpt-5
    for dataset in [{"minecraft": 1}, {"copypasta": 1}, {"ucla": 1}]:
        dataset_name = list(dataset.keys())[0]
        for train_size in [10, 100]:
            generated_posts = load_generated_posts("train_size_ablation", model, dataset_name, train_size)[:10]
            posts.extend(generated_posts)

# write judgments to file
with open("judgements.json", "w") as f:
    # combine posts and judgements
    for post, judgement in zip(posts, judgements):
        post["judgement"] = judgement
    json.dump(posts, f)

In [ ]:
import json

style_adherence_acc = 0
content_adherence_acc = 0
coherence_acc = 0

total_posts = len(posts)
with open("judgements-gpt5.json", "r") as f:
    judgements = json.load(f)

for post in judgements:
    print("title: ", post["title"])
    print("self_text: ", post["self_text"])
    print("subreddit: ", post["subreddit"])
    
    judgement = post["judgement"]
    style_adherence = input("style_adherence: ")
    content_adherence = input("content_adherence: ")
    coherence = input("coherence: ")
    
    style_adherence_acc += 1 if style_adherence == judgement["style_adherence"] else 0
    content_adherence_acc += 1 if content_adherence == judgement["content_adherence"] else 0
    coherence_acc += 1 if coherence == judgement["coherence"] else 0
    
    print(judgement, "\n\n")
    

style_adherence_acc /= total_posts
content_adherence_acc /= total_posts
coherence_acc /= total_posts

print(f"Style adherence accuracy: {style_adherence_acc}")
print(f"Content adherence accuracy: {content_adherence_acc}")
print(f"Coherence accuracy: {coherence_acc}")


## Judge all posts

I've decided not to use llm judge bc the reliability is too low. i have fully committed to doing nothing except read slop reddit posts and judge them for the next few days. yeehaw

In [ ]:
# train size ablation
from tqdm.auto import tqdm
import json

judgments = []
for model in tqdm(MODELS, desc="Models"): # cost: about $5 for 120 judgements
    if model == "gpt-5": continue # skip gpt-5
    for dataset_name, dataset in tqdm(zip(DATASET1_NAMES, DATASETS1), desc="Datasets"):
        for train_size in tqdm(TRAIN_SIZES[:6], desc="Train Sizes"): # 10, 20, 50, 100, 250, 500
            style_adherence_count = 0
            content_adherence_count = 0
            coherence_count = 0
            
            generated_posts = load_generated_posts("train_size_ablation", model, dataset_name, train_size)[:50] # only judge half of the posts bc i am short on time and money
            
            for post in tqdm(generated_posts, desc="Posts"):
                post = f"title: {post['title']}\n self_text: {post['self_text']}\n subreddit: {post['subreddit']}\n\n"
                judgement = json.loads(judge_post(post, dataset))

                style_adherence_count += judgement["style_adherence"]
                content_adherence_count += judgement["content_adherence"]
                coherence_count += judgement["coherence"]
                
            style_adherence = style_adherence_count / len(generated_posts)
            content_adherence = content_adherence_count / len(generated_posts)
            coherence = coherence_count / len(generated_posts)
            
            judgments.append({
                "model": model,
                "dataset_name": dataset_name,
                "train_size": train_size,
                "style_adherence": style_adherence,
                "content_adherence": content_adherence,
                "coherence": coherence
            })
# write to json
with open("judgements-train-size-ablation.json", "w") as f:
    json.dump(judgments, f)
                

## Cosine Similarity

In [ ]:
# word embedding cos similarity between posts in the generated dataset to see how varied the generated posts are
# TODO